# 1. Simulation

In [31]:
from string import ascii_uppercase
from draw_utils import *
from pyglet.gl import *
import numpy as np
import pandas as pd
import os



# reward
move_reward = - 0.09
obs_reward = - 0.1
goal_reward = 10
print('reward:' , move_reward, obs_reward, goal_reward)

local_path = '/home/zlxlekta924/hsm3' #os.path.abspath(os.path.join(os.path.dirname(__file__)))


class Simulator:
    def __init__(self):

        '''
        height : 그리드 높이
        width : 그리드 너비 
        inds : A ~ Q alphabet list
        '''
        # Load train data
        self.files = pd.read_csv(os.path.join(local_path, "./data/factory_order_train.csv"))
        self.height = 10
        self.width = 9
        self.inds = list(ascii_uppercase)[:17]
        self.clear_item = False

    def set_box(self):
        '''
        아이템들이 있을 위치를 미리 정해놓고 그 위치 좌표들에 아이템이 들어올 수 있으므로 그리드에 100으로 표시한다.
        데이터 파일에서 이번 에피소드 아이템 정보를 받아 가져와야 할 아이템이 있는 좌표만 -100으로 표시한다.
        self.local_target에 에이전트가 이번에 방문해야할 좌표들을 저장한다.
        따라서 가져와야하는 아이템 좌표와 end point 좌표(처음 시작했던 좌표로 돌아와야하므로)가 들어가게 된다.
        '''
        box_data = pd.read_csv(os.path.join(local_path, "./data/box.csv"))
        
        #물건이 들어있을 수 있는 곳
        for box in box_data.itertuples(index = True, name ='Pandas'):
            self.grid[0][getattr(box, "row")][getattr(box, "col")] = 0
            
        # 물건이 실제 들어있는 경우
        order_item = list(set(self.inds) & set(self.items))
        order_csv = box_data[box_data['item'].isin(order_item)]  
        
        for order_box in order_csv.itertuples(index = True, name ='Pandas'):
            self.local_target.append(
                [getattr(order_box, "row"),
                 getattr(order_box, "col")]
                )
            
        self.local_target.append([9,4]) #local_target : 우리가 가야할 목적지의 나열 list
        self.grid[1][self.local_target[0][0]][self.local_target[0][1]] = 1 #가장 첫번째에 1

    def set_obstacle(self):
        '''
        장애물이 있어야하는 위치는 미리 obstacles.csv에 정의되어 있다. 이 좌표들을 0으로 표시한다.
        '''
        self.grid[0] = np.ones((10,9), dtype="float16") # 일단 다 1로 설정하고
        
        obstacles_data = pd.read_csv(os.path.join(local_path, "./data/obstacles.csv"))
        for obstacle in obstacles_data.itertuples(index = True, name ='Pandas'):
            self.grid[0][getattr(obstacle, "row")][getattr(obstacle, "col")] = 0

    def reset(self, epi):
        '''
        reset()은 첫 스텝에서 사용되며 그리드에서 에이전트 위치가 start point에 있게 한다.

        :param epi: episode, 에피소드 마다 가져와야 할 아이템 리스트를 불러올 때 사용
        :return: 초기셋팅 된 그리드
        :rtype: numpy.ndarray
        _____________________________________________________________________________________
        items : 이번 에피소드에서 가져와야하는 아이템들
        terminal_location : 현재 에이전트가 찾아가야하는 목적지
        local_target : 한 에피소드에서 찾아가야하는 아이템 좌표, 마지막 엔드 포인트 등의 위치좌표들
        actions: visualization을 위해 에이전트 action을 저장하는 리스트
        curloc : 현재 위치
        '''

        # initial episode parameter setting
        self.epi = epi
        self.items = list(self.files.iloc[self.epi])[0]
        self.cumulative_reward = 0
        self.terminal_location = None
        self.local_target = []
        self.actions = []

        # initial grid setting
        self.grid = np.zeros((3,self.height, self.width), dtype="float16")#장애물, 아이템, 에이전트

        # set information about the gridworld
        self.set_obstacle()
        self.set_box()

        # start point를 grid에 표시
        self.curloc = [9, 4]
        self.grid[2][int(self.curloc[0])][int(self.curloc[1])] = 1


        self.done = False  
        

        

        
        return self.grid

    def apply_action(self, action, cur_x, cur_y):
        '''
        에이전트가 행한 action대로 현 에이전트의 위치좌표를 바꾼다.
        action은 discrete하며 4가지 up,down,left,right으로 정의된다.
        
        :param x: 에이전트의 현재 x 좌표
        :param y: 에이전트의 현재 y 좌표
        :return: action에 따라 변한 에이전트의 x 좌표, y 좌표
        :rtype: int, int
        '''
        new_x = cur_x
        new_y = cur_y
        
        # up
        if action == 0:
            new_x = cur_x - 1
        # down
        elif action == 1:
            new_x = cur_x + 1
        # left
        elif action == 2:
            new_y = cur_y - 1
        # right
        else:
            new_y = cur_y + 1

        return int(new_x), int(new_y)


    def get_reward(self, new_x, new_y, out_of_boundary):
        '''
        get_reward함수는 리워드를 계산하는 함수이며, 상황에 따라 에이전트가 action을 옳게 했는지 판단하는 지표가 된다.

        :param new_x: action에 따른 에이전트 새로운 위치좌표 x
        :param new_y: action에 따른 에이전트 새로운 위치좌표 y
        :param out_of_boundary: 에이전트 위치가 그리드 밖이 되지 않도록 제한
        :return: action에 따른 리워드
        :rtype: float
        '''

        # 바깥으로 나가는 경우
        if any(out_of_boundary):
            reward = obs_reward
                       
        else:
            # 장애물에 부딪히는 경우 
            if self.grid[0][new_x][new_y] == 0 and self.grid[1][new_x][new_y] == 0:
                reward = obs_reward  

            # 현재 목표에 도달한 경우
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:
                reward = goal_reward

            # 그냥 움직이는 경우 
            else:
                reward = move_reward

        return reward

    def step(self, action):
        ''' 
        에이전트의 action에 따라 step을 진행한다.
        action에 따라 에이전트 위치를 변환하고, action에 대해 리워드를 받고, 어느 상황에 에피소드가 종료되어야 하는지 등을 판단한다.
        에이전트가 endpoint에 도착하면 gif로 에피소드에서 에이전트의 행동이 저장된다.

        :param action: 에이전트 행동
        :return:
            grid, 그리드
            reward, 리워드
            cumulative_reward, 누적 리워드
            done, 종료 여부
            goal_ob_reward, goal까지 아이템을 모두 가지고 돌아오는 finish율 계산을 위한 파라미터

        :rtype: numpy.ndarray, float, float, bool, bool/str

        (Hint : 시작 위치 (9,4)에서 up말고 다른 action은 전부 장애물이므로 action을 고정하는 것이 좋음)
        '''

        self.terminal_location = self.local_target[0]
        
        cur_x,cur_y = self.curloc
        
        self.actions.append((cur_x, cur_y))

        goal_ob_reward = False
        
        new_x, new_y = self.apply_action(action, cur_x, cur_y)

        out_of_boundary = [new_x < 0, new_x >= self.height, new_y < 0, new_y >= self.width]

        # 바깥으로 나가는 경우
        if any(out_of_boundary):
            
            reward = self.get_reward(new_x, new_y, out_of_boundary)
            #self.done = True
            #goal_ob_reward = True
        
        
        #바깥으로 나가지 않는 경우
        else:
            
            # 장애물이 있고 아이템이 없음
            if self.grid[0][new_x][new_y] == 0 and self.grid[1][new_x][new_y] == 0: #장애물이고 아이템도 없음
                #self.done = True
                #goal_ob_reward = True
                reward = self.get_reward(new_x, new_y, out_of_boundary) ###############################수정



            # 아이템이 있음
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:

                                
  
                # 마지막 end point 일 때
                if [new_x, new_y] == [9,4]:
                    self.done = True
                

                #다음 장소가 있다면
                if self.local_target: 
                    self.grid[1][self.local_target[0][0]][self.local_target[0][1]] = 1
                
                self.local_target.remove(self.local_target[0]) #현재 장소 삭제
                
                #에이전트 위치 변경
                self.grid[2][cur_x][cur_y] = 0
                self.grid[2][new_x][new_y] = 1
                
                goal_ob_reward = True
                
                self.curloc = [new_x, new_y]
                
                reward = self.get_reward(new_x, new_y, out_of_boundary)
                
                #지금 아이템 장소 변경
                self.grid[1][self.terminal_location[0]][self.terminal_location[1]] = 0
                

                
            else:
                # 그냥 움직이는 경우 

                
                self.grid[2][cur_x][cur_y] = 0    
                self.grid[2][new_x][new_y] = 1
                    
                
                self.curloc = [new_x,new_y]
                
                reward = self.get_reward(new_x, new_y, out_of_boundary)
                
        self.cumulative_reward += reward

        
        
        #만약 끝났으면, 

        if self.done == True:
            if [new_x, new_y] == [9, 4]:
                if self.terminal_location == [9, 4]:
                    
                   
                    
                    print(f'Epi : {self.epi} \n')

                    if len(self.actions) <= 200:
     # 완료되면 GIFS 저장

                        goal_ob_reward = 'finish'
                        height = 10
                        width = 9 
                        display = Display(visible=False, size=(width, height))
                        display.start()

                        start_point = (9, 4)
                        unit = 50
                        screen_height = height * unit
                        screen_width = width * unit
                        log_path = "./logs_nostop"
                        data_path = "./data"
                        render_cls = Render(screen_width, screen_height, unit, start_point, data_path, log_path)
                        for idx, new_pos in enumerate(self.actions):
                            render_cls.update_movement(new_pos, idx+1)

                        render_cls.save_gif(self.epi)
                        render_cls.viewer.close()
                        display.stop()
                        print(f'Action num : \n{len(self.actions)}')
                        print(f'Action History : \n{self.actions}\n 그림 제작 완료.')
                    else:
                        print(f'Action num : \n{len(self.actions)}')
        
        return self.grid, reward, self.cumulative_reward, self.done, goal_ob_reward

    def now_state(self):
        print(f'Epi : {self.epi}, \n챙기는 item : {self.local_target}')
        return


reward: -0.09 -0.1 10


## 1. Agent 구성

Agent 구성에 필요한 요소

1. Replay Buffer
2. Qnet

In [2]:
import collections
import pdb
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#from Sim import *



/home/zlxlekta924/anaconda3/envs/test/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Replay buffer

In [3]:

## Replay buffer

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)

    def put(self, transition):
        self.buffer.append(transition)

    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)     #sample 메서드 : buffer 중에 n 개만 뽑음
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
                                                        # state 값, action 값, reward 값, 다음 state 값, done_mask 값
                                                        # done_mask : 종료 상태의 Value 값을 마스킹해줍니다

        for transition in mini_batch: #우리가 뽑은 n 개의 미니배치들 하나씩에서
            s, a, r, s_prime, done_mask = transition #

            # 이로 보아 하나의 sample 에는 s,a,s_prime,done_mask 값이 담김을 확인할 수 있다
            s_lst.append(s)
            a_lst.append([a]) # 자료형이 list 임을 확인 가능, 여러개의 action 이 담겨있을 것이라 추측
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch. float), torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
        # s_lst, s_prime_lst 는 타입을 바꿔주었다. 데이터가 int 였기 때문에
        # 나머지는 tensor화

    def size(self):
        return len(self.buffer)




### Qnet

In [12]:

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(9*10,128) # 배열데이터 9*10 근데 이게 2차원 데이터인데 그냥 넣어도 되나? #Flatten
        self.fc2 = nn.Linear(128,128)
        self.fc3 = nn.Linear(128,4) #상하좌우의 action state 값을 반환

    def forward(self,x):
        
        #pdb.set_trace()
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # 4개의 값을 반환
        return x

    def sample_action(self, obs, epsilon):
        out = self.forward(obs) ### obs 에 무엇이 들어가는가? 카트의 위치,속도, 막대의 각도, 각속도 4개 # Flatten
        coin = random.random() # 0~1 사이의 값을 랜덤하게 호출
        

        if coin < epsilon:  # 동전던지기, 8% 확률로 랜덤하게 행동
            return random.randint(0,3)
        else:
            return out.argmax().item() #Action state 값이 더 높은 index 값 호출
        


class QnetCNN(nn.Module):
    def __init__(self):
        super(QnetCNN, self).__init__()
        
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=5))

        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
    
        self.dropout = nn.Dropout()

        #Flatten
        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(128 , 256)
        self.fc2 = nn.Linear(256, 4)
        
    def forward(self,x):
        
        x = torch.reshape(x, (-1, 3, 10, 9)).to(device)

        #pdb.set_trace()
        x = self.layer1(x)
        x.shape
        x = self.layer2(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)# 4개의 값을 반환
        return x

    def sample_action(self, obs, epsilon):
        out = self.forward(obs) ### obs 에 무엇이 들어가는가? 카트의 위치,속도, 막대의 각도, 각속도 4개 # Flatten
        coin = random.random() # 0~1 사이의 값을 랜덤하게 호출
        

        if coin < epsilon:  # 동전던지기, 8% 확률로 랜덤하게 행동
            return random.randint(0,3)
        else:
            return out.argmax().item() #Action state 값이 더 높은 index 값 호출



## 해당 재료를 가지고 train


In [24]:

## 학습 함수

def train(q, q_target, memory, optimizer): 
    for i in range(10):
        s, a, r, s_prime, done_mask = memory.sample(batch_size) #32개를 버퍼에서 뽑아 모아 놓은 s,a,r,s_prime,done_mask

        q_out = q(s)                                   # s 값으로 다음 각 action 값들의 value 값 반환
        q_a = q_out.gather(1,a.to(device))         #선택한 액션값들의 q(s,a) 반환
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1) # 다음 state의 각 q(s,a) 값 반환
        target = torch.tensor(r, device=device) + torch.tensor(gamma, device=device) * max_q_prime * torch.tensor(done_mask, device=device) # 배열 맞춰주기, 쓰러진 경우는 제거
        loss = F.smooth_l1_loss(q_a, target) # DQN 의 손실함수 계산 L1 유클리드

        optimizer.zero_grad() # optimizer 의 모든 parameter 를 0으로 변환
        loss.backward() # loss 에 대한 gradient 계산
        optimizer.step() # 손실값을 바탕으로 Qnet 의 파라미터 업데이트


## Model save

In [6]:
def model_save(model_dict, opt_dict, epi):
    PATH = './weights/'
    torch.save({
            'model': model_dict,
            'optimizer': opt_dict,
            'epi': epi
            }, PATH + 'all.tar')

#### 전체적인 구성

In [ ]:
import time
from tqdm import tqdm

learning_rate = 0.0005
gamma = 0.99
buffer_limit  = 500000
batch_size = 32

#GPU 사용 : compile 또는 loss 연산에 사용 -> .to(device)
USE_CUDA = torch.cuda.is_available()
#print(USE_CUDA)
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

def main():
    
    env = Simulator()
    files = pd.read_csv("./data/factory_order_train.csv")

    q = QnetCNN().to(device)
    q_target = QnetCNN().to(device)
    q_target.load_state_dict(q.state_dict()) # 현재 Qnet 의 파라미터를 q_target 에 load

    memory = ReplayBuffer()

    print_interval = 50
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr = learning_rate) # loss 값을 바탕으로 업데이트할 비율 (q_target 말고 q 만 업데이트)

    

    for epi in tqdm(range(40000)):
        time.sleep(0.1)
        
        if epi%1000 == 0:
            model_save(q.state_dict(), optimizer.state_dict(), epi)
        
        items = list(files.iloc[epi%39980])[0]

        epsilon = max(0.01, 0.20 - 0.02 * (epi//200))
        # exploration 비율 - 2000번마다 1%씩 10%에서 1%로 감소

        s = env.reset(epi) # 재시작, epi 번째의 item list 가져옴
        
        ######################수정부분
        first = True
        
        a_step= 0
        
        obs = np.asarray(s, dtype=np.float32) #첫번째 state 값

        ###
        
        done = False 

        while not done: # 게임이 끝날 때까지

                
            if first:
                a = 0
                first = False
                
            else:
                a = q.sample_action(torch.from_numpy(obs).float(), epsilon) 
            
            s_prime, r, cumul, done, goal_reward = env.step(a) # a를 선택했을때
            
            
            if a_step ==100: #1000번 했는데도 안되면 강제종료
                done =True
                goal_reward=True
            
            array = s_prime
            
            action = ['↑', '↓', '←', '→'][a]
            
            s_prime = np.asarray(s_prime, dtype=np.float32)
             
            ###
            
            done_mask = 0.0 if done else 1.0 # False 가 아직 안끝난 경우, true 가 끝난경우

            memory.put((obs,a,r, s_prime, done_mask))

            obs = s_prime # 다음 s 값으로 이어서 진행
            score += r
        
                
            if done : # 끝났으면 train 안함
                break
                
                
            a_step +=1 # a 의 시행횟수


        if memory.size() > 4000: 
            train(q, q_target, memory, optimizer) 

        if epi%print_interval==0 and epi != 0: 
            print(f'epi = {epi}\n 마지막 상태 :\n{array}\n 이 때 한 행동 : {action}\n 마지막 보상 : {r}\n 총 받은 보상 :{cumul}\n end : {done}\n clear : {goal_reward}')
            env.now_state()
            
            q_target.load_state_dict(q.state_dict()) #q_target 지금걸로 업데이트
            print(f"episode :{epi}, score = {int(score/print_interval)}, n_buffer : {memory.size()}, eps : {epsilon*100}")
            score = 0.0

main()


  0%|                                                                     | 39/40000 [00:08<2:26:46,  4.54it/s]/home/zlxlekta924/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
  0%|                                                                     | 51/40000 [00:12<3:37:21,  3.06it/s]

epi = 50
 마지막 상태 :
[[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 1. 1. 1. 1. 1.]
  [0. 1. 1. 1. 1. 1. 1. 1. 0.]
  [0. 1. 0. 1. 0. 1. 0. 1. 0.]
  [0. 1. 0. 1. 0. 1. 0. 1. 0.]
  [0. 1. 0. 1. 0. 1. 0. 1. 0.]
  [1. 1. 0. 1. 0. 1. 0. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1. 1.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
 이 때 한 행동 : ←
 마지막 보상 : -0.1
 총 받은 보상 :-9.709

  0%|▏                                                                   | 101/40000 [00:29<3:48:37,  2.91it/s]

epi = 100
 마지막 상태 :
[[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 1. 1. 1. 1. 1.]
  [0. 1. 1. 1. 1. 1. 1. 1. 0.]
  [0. 1. 0. 1. 0. 1. 0. 1. 0.]
  [0. 1. 0. 1. 0. 1. 0. 1. 0.]
  [0. 1. 0. 1. 0. 1. 0. 1. 0.]
  [1. 1. 0. 1. 0. 1. 0. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1. 1.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
 이 때 한 행동 : →
 마지막 보상 : -0.1
 총 받은 보상 :-9.66

  0%|▎                                                                   | 151/40000 [00:46<3:29:06,  3.18it/s]

...

  7%|████▋                                                              | 2801/40000 [16:52<3:48:27,  2.71it/s]

epi = 2800
 마지막 상태 :
[[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 1. 1. 1. 1. 1.]
  [0. 1. 1. 1. 1. 1. 1. 1. 0.]
  [0. 1. 0. 1. 0. 1. 0. 1. 0.]
  [0. 1. 0. 1. 0. 1. 0. 1. 0.]
  [0. 1. 0. 1. 0. 1. 0. 1. 0.]
  [1. 1. 0. 1. 0. 1. 0. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1. 1.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
 이 때 한 행동 : ↑
 마지막 보상 : -0.1
 총 받은 보상 :-9.9

  7%|████▊                                                              | 2851/40000 [17:10<4:00:34,  2.57it/s]

epi = 2850
 마지막 상태 :
[[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 1. 1. 1. 1. 1.]
  [0. 1. 1. 1. 1. 1. 1. 1. 0.]
  [0. 1. 0. 1. 0. 1. 0. 1. 0.]
  [0. 1. 0. 1. 0. 1. 0. 1. 0.]
  [0. 1. 0. 1. 0. 1. 0. 1. 0.]
  [1. 1. 0. 1. 0. 1. 0. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1. 1.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]]]
 이 때 한 행동 : ↓
 마지막 보상 : -0.1
 총 받은 보상 :-10.

  7%|████▊                                                              | 2892/40000 [17:26<3:58:34,  2.59it/s]

## Saved model Using..

In [14]:
'''
import time
from tqdm import tqdm

learning_rate = 0.0005
gamma = 0.98
buffer_limit  = 100000
batch_size = 32

def main():
    
    env = Simulator()
    files = pd.read_csv("./data/factory_order_train.csv")

    q = QnetCNN()
    q_target = QnetCNN()
    optimizer = optim.Adam(q.parameters(), lr = learning_rate) # loss 값을 바탕으로 업데이트할 비율 (q_target 말고 q 만 업데이트)

    PATH = './weights/'
    checkpoint = torch.load(PATH+'all.tar')
    q.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epi_num = checkpoint['epi']
    #########################################
    
    q_target.load_state_dict(q.state_dict()) # 현재 Qnet 의 파라미터를 q_target 에 load

    memory = ReplayBuffer()

    print_interval = 100
    score = 0.0

    

    for epi in tqdm(range(epi_num, 200000)):
        time.sleep(0.1)
        
        if epi%1000 == 0:
            model_save(q.state_dict(), optimizer.state_dict(), epi)
        
        items = list(files.iloc[epi%50000])[0]

        epsilon = max(0.01, 0.20 - 0.02 * (epi//200))
        # exploration 비율 - 2000번마다 1%씩 10%에서 1%로 감소

        s = env.reset(epi) # 재시작, epi 번째의 item list 가져옴
        
        ######################수정부분
        first = True
        
        a_step= 0
        
        obs = np.asarray(s, dtype=np.float32) #첫번째 state 값

        ###
        
        done = False 
        while not done: # 게임이 끝날 때까지

            if first:
                a = 0
                first = False
                
            else:
                a = q.sample_action(torch.from_numpy(obs).float(), epsilon) 
            
            s_prime, r, cumul, done, goal_reward = env.step(a) # a를 선택했을때
            
            
            if a_step ==1000: #100번 했는데도 안되면 강제종료
                done =True
                goal_reward=True
            
            array = s_prime
            
            action = ['↑', '↓', '←', '→'][a]
            
            s_prime = np.asarray(s_prime, dtype=np.float32)
             
            ###
            
            done_mask = 0.0 if done else 1.0 # False 가 아직 안끝난 경우, true 가 끝난경우

            memory.put((obs,a,r, s_prime, done_mask))

            obs = s_prime # 다음 s 값으로 이어서 진행
            score += r
        
                
            if done : # 끝났으면 train 안함
                break
                
                
            a_step +=1 # a 의 시행횟수

            


        if memory.size() > 2000: 
            train(q, q_target, memory, optimizer) 

        if epi%print_interval==0 and epi != 0: 
            print(f'epi = {epi}\n 마지막 상태 :\n{array}\n 이 때 한 행동 : {action}\n 마지막 보상 : {r}\n 총 받은 보상 :{cumul}\n end : {done}\n clear : {goal_reward}')

            q_target.load_state_dict(q.state_dict()) #q_target 지금걸로 업데이트
            print(f"episode :{epi}, score = {int(score/print_interval)}, n_buffer : {memory.size()}, eps : {epsilon*100}")
            score = 0.0

main()
'''

NameError: name 'QnetCNN' is not defined